# Mouthpiece Neural Network
Initialization of modules

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers

model = Sequential()

Using TensorFlow backend.


Network Parameters

In [2]:
model.add(Dense(units=128, activation='elu', input_dim=500))
model.add(Dropout(0.1))
model.add(Dense(units=128, activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(units=12, activation='softmax'))

Network Compilation

In [3]:
opt = optimizers.rmsprop(lr=0.1)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Training Time!!

In [4]:
import scipy.io.wavfile as wv
import numpy as np
import os
from keras.utils import to_categorical

dat = list()
label = list()

for root, d, f in os.walk('..\data\Voices'):
    for file in f:
        r, data = wv.read(root + '\\' + file)
        dirs = root.split('\\')

        new_dat = list()
        for amp in data:
            if isinstance(amp,np.ndarray):
                new_dat.append(amp[0])
            else:
                new_dat.append(amp)

        data = np.array(new_dat)
        rate = int(r * 0.020)
        n_segs = int(data.shape[0]/rate)

        for i in range(n_segs):
            start = i * rate
            end = start + rate
            dat_seg = data[start:end]
            dat.append(dat_seg)
            label.append(dirs[3])


d:\documents\tuks\cos 301\mini\neuralnetwork\venv\lib\site-packages\ipykernel_launcher.py:11: WavFileWarning: Chunk (non-data) not understood, skipping it.
  # This is added back by InteractiveShellApp.init_path()


In [5]:
dat500 = []
for item in dat:
    div, rem = divmod(len(item),500)
    dat250it = []
    for i in range(rem):
        start = i*(div+1)
        end = start + div + 1
        tot = sum(item[start:end])
        ave = divmod(tot,div+1)[0]
        dat250it.append(ave)

    rest = 500 - rem
    for i in range(rest):
        start = i*div
        end = start + div
        tot = sum(item[start:end])
        ave = divmod(tot,div)[0]
        dat250it.append(ave)
    dat250it = np.array(dat250it)
    dat500.append(dat250it)

dat500 = np.array(dat500)

In [6]:
label_enc = to_categorical(label)

In [9]:
model.fit(dat500, label_enc, batch_size=16, epochs=20, verbose=1)

Epoch 1/20
20863/20863 [==============================] - 3s 161us/step - loss: 49.9159 - accuracy: 0.1208

In [10]:
model.evaluate(dat500, label_enc, verbose=1)

20863/20863 [==============================] - 1s 46us/step

[23.352648594667063, 0.25528448820114136]

In [11]:
model.save_weights('../models/voice.h5')
